<a href="https://colab.research.google.com/github/sonewn/wanted_pre_onboarding/blob/main/wanted_pre_onboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from math import log
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 문제 1) Tokenizer 생성하기




In [2]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False
  
    def preprocessing(self, sequences):
        result = []
        '''
        문제 1-1.
        '''
        for seq in sequences:
            word_seq = text_to_word_sequence(seq)
            result.append(word_seq)
        return result
  
    def fit(self, sequences):
        self.fit_checker = False
        '''
        문제 1-2.
        '''
        tokens = self.preprocessing(sequences)
        base = []
        for tok in tokens:
            base.extend(tok)
        uni_token = pd.Series(base).unique()
        key_lst = list(uni_token)

        for idx, key in zip(range(len(key_lst)), key_lst):
            self.word_dict[key] = idx+1
        self.fit_checker = True
  
    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            '''
            문제 1-3.
            '''
            for tok in tokens:
                tokidx = [self.word_dict[i] for i in tok]
                result.append(tokidx)
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
      
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

<h3> &nbsp; ✅ 실행 결과 확인




In [3]:
tokenizer = Tokenizer()
lst =  ['I go to school.', 'I LIKE pizza!']
tokenizer.fit_transform(lst)

[[1, 2, 3, 4], [1, 5, 6]]

# 문제 2) TfidfVectorizer 생성하기

In [8]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
        self.idf_lst = []
        self.tfidf_matrix = []
        self.final_token = -1
  
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        '''
        문제 2-1.
        '''
        # 마지막 token 값 찾기
        final_tok = -1
        for tok in tokenized:
            tok_max = max(tok)
            final_tok = max(final_tok, tok_max)
        self.final_token = final_tok

        # token 별 idf 구하기
        for no in range(final_tok):
            n_n = len(tokenized)
            df = 0
            for tok in tokenized:
                df += (no+1) in tok
            idf = log(n_n/(1+df))
            self.idf_lst.append(idf)
        self.fit_checker = True
    

    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            '''
            문제 2-2.
            '''
            # 문장 별 tf 생성
            tf_lst = []
            for tok in tokenized:
                tf = [tok.count(idx+1) for idx in range(self.final_token)]
                tf_lst.append(tf)
            
            # tf-idf matrix 생성
            for tf in tf_lst:
                tf = np.array(tf)
                idf = np.array(self.idf_lst)
                self.tfidf_matrix.append(list(tf*idf))
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")


    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

<h3> &nbsp; ✅ 실행 결과 확인


In [9]:
tfid = TfidfVectorizer(tokenizer)
tfid.fit_transform(lst)

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]]